In [6]:
import qsonar.load

In [7]:
# Load data from the internet (or from local file if already downloaded) into pandas DataFrame.
# Features: f0, ..., f59 as np.float64
# Target: Y as 'M'/'R' (mine/rock)

df = qsonar.load.load_data_from_internet()

In [19]:
# Scale and split the data into training and test sets.
# X_train, X_test: np.ndarray of shape (m, n)s of np.float64 data
# y_train, y_test: np.ndarray of booleans of shape (m,) where True means 'M'ine and False means 'R'ock
# Default split is 80% training and 20% test and default random seed is 1337.

X_train, X_test, y_train, y_test = qsonar.load.scale_and_split(df)

In [20]:
X_train = X_train[:100, :8]
X_test = X_test[:100, :8]
y_train = y_train[:100]
y_test = y_test[:100]

In [21]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit_machine_learning.algorithms import QSVC
from qiskit.circuit.library import ZZFeatureMap
from qiskit.primitives import Sampler
from qiskit_algorithms.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from sklearn.metrics import accuracy_score

# Initialize the AerSimulator
sim = AerSimulator()

# Define the feature map
feature_map = ZZFeatureMap(feature_dimension=4, reps=2, entanglement='linear')

# Define the fidelity quantum kernel
sampler = Sampler()
fidelity = ComputeUncompute(sampler=sampler)
quantum_kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=feature_map)

# Train the QSVC
qsvc = QSVC(quantum_kernel=quantum_kernel)
qsvc.fit(X_train, y_train)

# Predict using the trained QSVC
Y = qsvc.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, Y)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.52
